In [2]:
import chromadb
from chromadb.utils import embedding_functions
import platform
from diarize import token_count

platform.platform()

path = (
    "/Users/ansel/chromadb"
    if "macOS" in platform.platform()
    else "/home/ansel/chromadb"
)

chroma_client = chromadb.PersistentClient(path=path)
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-mpnet-base-v2"
)

collection = chroma_client.get_collection(
    name="roderick", embedding_function=sentence_transformer_ef
)

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:datasets:PyTorch version 2.3.0 available.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2


In [15]:
raw_results = collection.query(
    query_texts=["What does John Roderick think about donuts?"],
    n_results=30,
    include=["documents", "metadatas"],
)

results = list(zip(raw_results["metadatas"][0], raw_results["documents"][0]))

formatted_results = [
    f"{metadata["podcast"]} - {metadata["title"]}, {metadata["date"]}\n\n{document}"
    for metadata, document in results
]

text = "\n\n".join(formatted_results)

context_input = (
    "Based on the following podcast excerpts, what does John Roderick think about donuts? Formlate your response in the voice of John Roderick."
    + "\n\n"
    + text
)

f = open("question_donuts_extended.txt", "w")
f.write(context_input)
f.close()

count = token_count(context_input)

count

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

6290